In [7]:
from core.libs import Path, openpyxl, os, pd
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Ruta base (ajústala según tu país/año/lote)
BASE_PATH = Path(r"D:\OneDrive Local\OneDrive - World Tree Technologies Inc\Forest Inventory - Documentos\USA\2024_ForestInventoryQ1_25")

In [8]:
# Acumular errores
errores = []

# Buscar todos los .xlsx recursivamente
xlsx_files = list(BASE_PATH.rglob("*.xlsx"))

In [9]:
for file in tqdm(xlsx_files, desc="📂 Scanning OneDrive files"):
    try:
        wb = openpyxl.load_workbook(file, data_only=True)
        if "Summary" not in wb.sheetnames:
            errores.append((file, "❌ Missing 'Summary' sheet"))
            continue

        sheet = wb["Summary"]
        d3_value = sheet["D3"].value

        if not d3_value or not isinstance(d3_value, str) or d3_value.strip().lower() == "contract code":
            errores.append((file, f"⚠️ Invalid value in D3: '{d3_value}'"))

    except Exception as e:
        errores.append((file, f"💥 Error reading file: {e}"))



📂 Scanning OneDrive files: 100%|██████████| 64/64 [00:19<00:00,  3.33it/s]


In [10]:
# Crear DataFrame
df = pd.DataFrame([
    {
        "Error": msg,
        "Filename": Path(path).name,
        "Directory Path": str(Path(path).parent)
    }
    for path, msg in errores
])

# Guardar como Excel
output_path = Path.cwd() / "malformed_d3a_entries.xlsx"
df.to_excel(output_path, index=False)

print(f"\n✅ Report saved to: {output_path}")


✅ Report saved to: D:\LOCAL\WorldTreeSystem\AuxiliaryScripts\malformed_d3a_entries.xlsx
